# Exercise Sheet 3


In [23]:
import matplotlib.pyplot as plt
import numpy as np
import time
import sklearn as skl
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report

### Clustering

We are using again the MNIST data from Exercise 3. The following should just access the data
you have already downloaded (adjust the data_home path!):

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml(name='mnist_784', data_home='/home/jaeger/Data')

**Exercise 4:** We are exploring several clustering techniques with the MNIST data.

**a** Perform *k*-means clustering with *k*=10 on this dataset. 
<ul>Using visualization techniques analogous to what we have done in the `Clustering` notebook for the faces data, can you determine the 'nature' of the 10 constructed clusters? Do the clusters (roughly) coincide with the 10 different actual digits? </ul>
<ul>Perform a supervised clustering evaluation using adjusted rand index. </ul>
<ul>Are the clustering results stable, when you perform several random restarts of *k*-means? You can use the rand index also for this: take the cluster labels produced by two runs of *k*-means as the input for the rand index. The rand index then measures how much the two clusterings coincide. </ul>

**b** Now perform hierarchical clustering on the data (in order to improve visibility in the constructed dendrograms, you can also use a much reduced dataset as constructed using `sklearn.utils.resample` shown below). Does the visual analysis of the dendrogram indicate a natural number of clusters? Using different cluster distance metrics (ward,single,average), what do the clusterings look like that are producec at the level of *k*=10 clusters? See the `Clustering` notebook for the needed Python code, including the `fcluster` method to retrieve 'plain' clusterings from the hierarchical clustering. 

Some investigations may be easier to perform (and results easier interpreted) if you use a smaller subset of the MNIST data. The following code shows how one can use the skl.utils.resample method for this:

In [3]:
## Exercise: Perform *k*-means clustering with *k*=10 on this dataset

from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, adjusted_rand_score
from sklearn.datasets import fetch_openml
import numpy as np

# Load MNIST data
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target.astype(int)  # Load features and integer labels

# Normalize pixel values
X = X / 255.0

# Optionally, split into training and testing sets (if desired)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Fit KMeans on the training data
n_clusters = 10  # Number of clusters (digits 0-9)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X_train)

# Assign clusters to the training and test data
train_clusters = kmeans.predict(X_train)
test_clusters = kmeans.predict(X_test)

# Evaluate clustering on training set (mapping clusters to labels for clarity)
from scipy.stats import mode

# Map each cluster to the most common label in it
labels = np.zeros_like(train_clusters)
for i in range(n_clusters):
    mask = (train_clusters == i)
    labels[mask] = mode(y_train[mask])[0]

# Calculate accuracy for the training data
train_accuracy = accuracy_score(y_train, labels)

print(f"Train Clustering Accuracy: {train_accuracy * 100:.2f}%")


Train Clustering Accuracy: 61.64%


In [ ]:
small_mnist_data,small_mnist_label = skl.utils.resample(mnist.data,mnist.target,n_samples=200,replace=False)

### Text data

We first use the dataset containing movie reviews from the *Internet Movie Database*. For this, the data first needs to be downloaded from <a href="http://ai.stanford.edu/~amaas/data/sentiment/"> here </a>. Note: this is about 220 Mb.  After uncompressing, the data is contained in a directory `aclImdb` with sub-directories `train` and `test`. In the following, replace the piece of the path that leads to the directory in which you have unpacked the data.

In [4]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

In [ ]:
reviews_train=load_files(r'C:\Users\KennethElong(KEEL)\Downloads\aclImdb_v1\aclImdb\train',categories=['neg','pos'])
reviews_test=load_files(r'C:\Users\KennethElong(KEEL)\Downloads\aclImdb_v1\aclImdb\test',categories=['neg','pos'])

The following is the same code as in the `TextData` notebook for transforming the data into term-frequency, and term-frequency-inverse-document-frequency vectors. 

In [19]:
## Splitting the training data into a training and validation set with 20% set aside for validation
X_train, X_val, y_train, y_val = train_test_split(
    reviews_train.data, reviews_train.target, test_size=0.2, random_state=11
)

# Vectorize the data
dictionary = CountVectorizer(min_df=0.0005, max_df=0.5).fit(X_train)

# Transform training, validation, and test data to TF-IDF
# Training data
reviews_train_tf_vec = dictionary.transform(X_train)
reviews_train_tfidf_vec = TfidfTransformer().fit_transform(reviews_train_tf_vec)

# Validation data
reviews_val_tf_vec = dictionary.transform(X_val)
reviews_val_tfidf_vec = TfidfTransformer().fit_transform(reviews_val_tf_vec)

# Test data
reviews_test_tf_vec = dictionary.transform(reviews_test.data)
reviews_test_tfidf_vec = TfidfTransformer().fit_transform(reviews_test_tf_vec)

**Exercise 5:** 

Learn at least 2 different kinds of classifiers (k-NN, Naive Bayes, Decision Tree, SVM, Neural Network) for predicting the 'positive' reviews based on the tf or tf-idf vectors. Using a validation set, optimize the classifiers via the available regularization parameters so that overfitting is minimized.  Evaluate the learned classifiers using accuracy, precision, recall, and their ROC curves. 


In [ ]:
## Printing the words at position 6951 to 6956
print(dictionary.get_feature_names_out()[6951:6956])

## This output now is not as it was in the TextData exercise, because the data used to build the dictionary was different


['heralded' 'herbert' 'hercules' 'herd' 'here']


Training a Naive Bayes model as the first one.

In [25]:
## Training the model
mnb=MultinomialNB().fit(reviews_train_tf_vec,y_train)

# Evaluate on the validation set
val_predictions = mnb.predict(reviews_val_tf_vec)
val_accuracy = accuracy_score(y_val, val_predictions)
print("Validation Accuracy: {:.4f}".format(val_accuracy))
#print("\nValidation Classification Report:\n", classification_report(y_val, val_predictions))

# Evaluate on the test set
test_predictions = mnb.predict(reviews_test_tf_vec)
test_accuracy = accuracy_score(reviews_test.target, test_predictions)
print("Test Accuracy: {:.4f}".format(test_accuracy))
#print("\nTest Classification Report:\n", classification_report(reviews_test.target, test_predictions))

Validation Accuracy: 0.8404
Test Accuracy: 0.8256


The Naive Bayers model predict 82.56% on the test data and 84,04% on the validation data. This seems like a fine balance between prediction on the validation and test set.

Opportunities to further enhance the model
- Remove stop words such as "the", "is" and "and"
- Experiment with the min_df and max_df to include or exclude more word at each end of the spectrum
- Adjust the alpha value, default set to 1, to see if this changes the outcome

Training a Support Vector Machine (SVM) model as the second